In [1]:
import pandas as pd
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import en_core_web_sm
from sklearn.metrics.pairwise import cosine_similarity
import re
nlp = en_core_web_sm.load()

import warnings
warnings.filterwarnings("ignore")

outfit = pd.read_csv('https://dso-560-nlp-text-analytics.s3.amazonaws.com/outfit_combinations.csv')
full_data = pd.read_csv('https://dso-560-nlp-text-analytics.s3.amazonaws.com/Full+data.csv')

### This is our input value

In [2]:
input_value = 'black leather jacket'

#### Imported dataset

In [3]:
outfit

,outfit_id,product_id,outfit_item_type,brand,product_full_name
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,Tory Burch,Penelope Mid Cap Toe Pump
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
...,...,...,...,...,...
5286,01E6MC52DNZE5AZ9MPZ2CYCXW9,01E5ZYHZA7186DVWEJ99Q4D2PM,accessory2,Sam Edelman,65mm Gradient Oversize Square Sunglasses
5287,01E6MC52DPCTAAQVYNSKA7PER7,01E2P0SJSKFKNQJ5SVQ8MD1JZT,shoe,DR. MARTENS,Fenimore Triple Buckle Boot
5288,01E6MC52DPCTAAQVYNSKA7PER7,01E4RW25Y8ZF6WKZRE50Y6SKH5,onepiece,Nili Lotan,Pamela Dress
5289,01E6MC52DPCTAAQVYNSKA7PER7,01E5ZS3R9JD696YWGK9NSG56E1,accessory1,Mansur Gavriel,Leather Circle Crossbody Bag


In [4]:
full_data = full_data.loc[:,['product_id', 'description']]
full_data

,product_id,description
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,Vintage Fitness leather sneakers with logo pri...
1,01DSQXJBX0R7DCW7KTAC1SW547,NaN
2,01DPGV8TGRAB993PF7Z3YWG2VR,A Timeless Leather Belt Crafted From Smooth Co...
3,01DSR8G3F7DBRTMP8THF97XSQ2,Pretty ruffle sleeves and trim elevate essenti...
4,01DSR8G5GP519DEDCSKBMWQVK5,Versatile convertible gown with elephant applique
...,...,...
42368,01DT5110V2ME7BY3JPCAJ91QRW,Mélange beige and cream wool Button fastenings...
42369,01DV6M2FXMPW9RSZWSXW1EK75W,"Cream georgette Ties at neck, concealed hook f..."
42370,01DV71D0XMBM4VVJK54F2HD3ZG,Sand cotton-corduroy Concealed hook and zip fa...
42371,01DV72R28K2A1AN5G167S6QRWF,Although mom jeans and boyfriend jeans are all...


#### I combined the two dataset through inner join

In [5]:
combined = pd.merge(outfit,
                    full_data,
                    on = 'product_id')
combined

,outfit_id,product_id,outfit_item_type,brand,product_full_name,description
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt,A nice skirt
1,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt,A nice skirt
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank,A nice tank
3,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank,A nice tank
4,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel,A nice bag
...,...,...,...,...,...,...
4776,01E6M4W99KF80RD4N7FFCW87SM,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,Vince,Belted Crepe Wide-Leg Pants,"Tailored in a leg-lengthening silhouette, Vinc..."
4777,01E6M4W99M1NSW0CJS736M1EJ6,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,Vince,Belted Crepe Wide-Leg Pants,"Tailored in a leg-lengthening silhouette, Vinc..."
4778,01E6M4W99M8GRP5ZXJ3FT1903G,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,Vince,Belted Crepe Wide-Leg Pants,"Tailored in a leg-lengthening silhouette, Vinc..."
4779,01E6M4W99MK8AHY989Y9Z99AQ8,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,Vince,Belted Crepe Wide-Leg Pants,"Tailored in a leg-lengthening silhouette, Vinc..."


In [6]:
input_value = input_value.upper()
if combined.loc[combined['product_id']==input_value,'product_id'].any() == input_value:
    input_descr = combined.loc[combined['product_id']==input_value,'description'].values[0]
else:
    input_descr = input_value

In [7]:
input_value

'BLACK LEATHER JACKET'

In [8]:
input_descr

'BLACK LEATHER JACKET'

In [9]:
combined.loc[combined['product_id']==input_value,:].head(1)

,outfit_id,product_id,outfit_item_type,brand,product_full_name,description


### Data Preprocessing

In [10]:
col = ['outfit_item_type', 'brand', 'product_full_name', 'description']

for columns in col:
    combined[columns] = combined[columns].str.lower() #making columns lowercase
combined.sort_values(by='outfit_id', inplace = True)
combined.reset_index(drop = True, inplace = True)

In [11]:
combined

,outfit_id,product_id,outfit_item_type,brand,product_full_name,description
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,eileen fisher,slim knit skirt,a nice skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,eileen fisher,rib mock neck tank,a nice tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel,a nice bag
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,tory burch,penelope mid cap toe pump,a nice shoe
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,eileen fisher,slim knit skirt,a nice skirt
...,...,...,...,...,...,...
4776,01E6M4W99MWGJ4434JNRME31NB,01DTATGYW72E9WEZHCKJBDFTCT,shoe,carrie forbes,bou raffia heeled mule,"for the past few seasons, mules have replaced ..."
4777,01E6M4W99NHRZGBVF7JJDEA4S7,01DTATGYW72E9WEZHCKJBDFTCT,shoe,carrie forbes,bou raffia heeled mule,"for the past few seasons, mules have replaced ..."
4778,01E6M4W99NZZ3DQ82FDWFHHH0J,01DTATGYW72E9WEZHCKJBDFTCT,shoe,carrie forbes,bou raffia heeled mule,"for the past few seasons, mules have replaced ..."
4779,01E6M4W99NZZ3DQ82FDWFHHH0J,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,vince,belted crepe wide-leg pants,"tailored in a leg-lengthening silhouette, vinc..."


In [12]:
# Defining the stopwords

stopwords_gensim = list(STOPWORDS)
stopwords_NLTK = list(stopwords.words("english"))
stopwords_combined = list(set(stopwords_gensim+stopwords_NLTK)) #to remove duplicates
negatives = ['not','nor','no','neither', 'never', 'bottom', 'top'] #took out the negative words for a more accurate analysis
stopwords_combined = list(filter(lambda x: x not in negatives, stopwords_combined))
stopwords_combined.sort()
stopwords_expression = '|'.join(stopwords_combined)
stopwords_pattern = f'({stopwords_expression})'

In [13]:
# Creating a stemming function

def stem_text(text):
    porter=PorterStemmer()
    tokens = text.split()
    stemmed_tokens = [porter.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [14]:
# Other data preprocessing for the description column

combined['description'] = combined['description'].astype(str) #making everythin string
combined['description'] = combined['description'].str.replace(r'[^\w\s]',' ') #replacing everything that is not a word with ""
combined['description'] = combined['description'].str.replace(rf'\b{stopwords_pattern}\b','') #removing stopwords
combined['description'] = combined['description'].apply(stem_text) #performing stemming

### Finding the most similar description to the input value

In [15]:
#Vectorizing the entire description column
description = list(combined.description) #making the description into a list
description = [str(i) for i in description] #making all the elements into string
description = list(set(description))
description_vectors = []
for i in description:
    temp_description = nlp(i)
    description_vectors.append(temp_description.vector)

In [16]:
# Pre-processing the input value just like the description column

input_descr = input_descr.lower() 
input_descr= re.sub(r'[^\w\s]',' ',input_descr)
input_descr = re.sub(rf'\b{stopwords_pattern}\b','',input_descr)
input_descr = stem_text(input_descr)


description.append(input_descr)

input_vectors = []
temp_input = nlp(input_descr)
description_vectors.append(temp_input.vector) #vectorizing and appending it to the description vectors

In [17]:
# Calculating the similariry score
vector_df =pd.DataFrame(description_vectors)
vector_df["description"] = description

vector_df.set_index("description", inplace=True)

from sklearn.metrics.pairwise import cosine_similarity

similarities = pd.DataFrame(cosine_similarity(vector_df.values), columns=description, index=description)

top_similarities = similarities.unstack().reset_index()
top_similarities.columns = ["input_descr", "original_description", "similarity"]
top_similarities = top_similarities.sort_values(by="similarity", ascending=False)
top_similarities = top_similarities[top_similarities["similarity"] <= 1]

In [18]:
top_similarities

,input_descr,original_description,similarity
26960,alexi crepe chine nari mini dress design puf s...,alexi crepe chine nari mini dress design puf s...,1.000000
359242,want sandal strappi support person thank mirro...,want sandal strappi support person thank mirro...,1.000000
244662,grlfrnd denim stretch denim kendal jean skinni...,grlfrnd denim stretch denim kendal jean skinni...,1.000000
290494,featur resort 20 lookbook ultra feminin zimmer...,featur resort 20 lookbook ultra feminin zimmer...,1.000000
72792,armi green stretch cotton corduroy button fast...,armi green stretch cotton corduroy button fast...,1.000000
...,...,...,...
415036,black pvc zip fasten 100 polyurethan back 100 ...,brush aesthet shirt exud retro vibe,0.257402
40323,sak list lux denim,black pvc zip fasten 100 polyurethan back 100 ...,0.223023
414627,black pvc zip fasten 100 polyurethan back 100 ...,sak list lux denim,0.223023
414897,black pvc zip fasten 100 polyurethan back 100 ...,nan,0.170205


In [19]:
# from the dataset above, I'm searching for the description that is most similar to the input value

match = top_similarities.loc[top_similarities['input_descr']== input_descr,:].reset_index(drop = True)
match

,input_descr,original_description,similarity
0,black leather jacket,cut cinch waist design leather jacket frame un...,0.830897
1,black leather jacket,prada croc effect flat emboss leather style cl...,0.830628
2,black leather jacket,design sleeveless lace bodic givenchi dress cr...,0.812656
3,black leather jacket,took classic button silhouett turn ultra flatt...,0.810595
4,black leather jacket,sleek faux leather midi skirt featur chic wrap...,0.808649
...,...,...,...
667,black leather jacket,cream crepe slip 57 acet 43 viscos line 96 pol...,0.525694
668,black leather jacket,camel cotton blend corduroy button conceal zip...,0.517694
669,black leather jacket,black mesh stretch polyest slip fabric1 88 rec...,0.512940
670,black leather jacket,black pvc zip fasten 100 polyurethan back 100 ...,0.442144


In [20]:
# using the output above, I could identify unique 3 products 

input_item = match.loc[0,'original_description']
output = combined.loc[combined.description == input_item,:]

for i in range(1,len(match)):
    match_item = match.loc[i,'original_description']
    match_combined = combined.loc[combined.description == match_item,:]
    output = pd.concat([output, match_combined])
    if output.product_id.nunique() >= 3:
        break
    else:
        continue

output.drop_duplicates(subset= ['outfit_item_type'], keep = 'first', inplace=True)
output.drop_duplicates(subset= ['product_id'], keep = 'first', inplace=True)
output.reset_index(drop = True, inplace = True)

In [21]:
# Next I'm matching the result to the original dataset to return the final recommendation result in original form.


product_list = []
for i in range(0, len(output)):
    product_list.append(output.loc[i,'product_id'])

In [22]:
data = pd.merge(outfit,
                    full_data,
                    on = 'product_id')

if data.loc[data['product_id']==input_value,'product_id'].any() == input_value:
  
    recommendation_output = data.loc[data['product_id']==input_value,:].head(1)

    recommendation = []
    for i in product_list:
        recommendation.append(data.loc[data.product_id == i,:])    

    recommendation = pd.concat(recommendation)
    recommendation.drop_duplicates(keep = 'first', subset= ['product_id'], inplace=True)
    recommendation.reset_index(drop = True, inplace = True)
    recommendation_output = recommendation_output.append(recommendation)

else:
    recommendation_output = []
    for i in product_list:
        recommendation_output.append(data.loc[data.product_id == i,:])    

    recommendation_output = pd.concat(recommendation_output)
    recommendation_output.drop_duplicates(keep = 'first', subset= ['product_id'], inplace=True)
recommendation_output.reset_index(drop = True, inplace = True)   
recommendation_output = recommendation_output.iloc[:,1:6]

In [23]:
recommendation_output

,product_id,outfit_item_type,brand,product_full_name,description
0,01DT0DHR8X5GZYR1KVBWAT3END,accessory2,Elie Tahari,Madison Single-Breasted Leather Jacket,"Cut into a cinched waist design, this leather ..."
1,01DVMH42BR0CRG0T9CVPD6EJ6M,shoe,Prada,Croc-Effect Leather Point-Toe Flats,Prada's croc-effect flats are made from emboss...
2,01DVA54THCXNRKYC0YVMWGYPJ5,onepiece,Givenchy,Lace-Paneled Velvet Mini Dress,"Designed with a sleeveless lace bodice, Givenc..."
